# CURRICULUM VITAE
## Luca Fontanili

In [1]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, LabelSet, RadioButtonGroup, CheckboxGroup
from bokeh.transform import linear_cmap
from bokeh.palettes import RdPu9
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row 
from bokeh.models.callbacks import CustomJS

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
from nltk.tokenize import word_tokenize
import string

with open('setup.html') as f:
    display(HTML(f.read()))

### DOWNLOAD A COPY

In [2]:
with open('download.html') as f:
    display(HTML(f.read()))

PDF,HTML


## PERSONAL INFORMATION

In [3]:
def load_map_frame():
    with open('map_frame_new.html') as f:
        return f.read()

import sqlite3
conn = sqlite3.connect('cv.db')
lf = pd.read_sql_query('SELECT * FROM users WHERE id="luca fontanili"', conn)
lf = lf.set_index('id')
display(HTML('<center><table><tr><td><img src="profile.jpg" width=230/>{}</td><td>{}</td></table></center>'
             .format(lf.T.to_html(escape=False), load_map_frame())))

<center>
<div id="banner">
    <div style="display: inline-block;">
        <a href="https://github.com/lucafon"><img src ="github.png" width=50></a>
    </div>
    <div style="display: inline-block;">
        <a href="https://www.linkedin.com/in/luca-fontanili"><img src ="linkedin.png" width=50></a>
    </div>
    <div style="display: inline-block;">
        <a href="https://pypi.org/user/lucafon"><img src ="pypi.png" width=100></a>
    </div>
</div>
</center>

_______________
## WORK EXPERIENCE


In [4]:
working_experiences = pd.read_sql_query('SELECT * FROM work_experiences where employer in ("Ubiq", "MyInvenio", "Cedacri", "Datalogic")', conn)
descriptions = working_experiences.drop(['employer', 'keyword'], axis=1)
keywords = working_experiences.drop(['employer', 'description'], axis=1)
employers = working_experiences.drop(['description', 'keyword'], axis=1)

def split_and_clean(input_string):
    return word_tokenize(str(input_string).lower().translate(str.maketrans('', '', string.punctuation)))

words = []
for idx in range(0, len(descriptions)):
    
    word_list = [word for word in split_and_clean(descriptions.iloc[idx]['description']) if not word in stopwords.words()]
    word_list += ([word for word in split_and_clean(keywords.iloc[idx]['keyword']) if not word in stopwords.words()])
    words.append(word_list)

hvec = HashingVectorizer(lowercase=False, analyzer=lambda l:l, n_features=2**10)
X = hvec.fit_transform(words)
tsne = TSNE(perplexity=5, method='exact')
X_embedded = tsne.fit_transform(X.toarray())

working_experiences['x'] = X_embedded[:,0]
working_experiences['y'] = X_embedded[:,1]

k = 7
kmeans = KMeans(n_clusters=k, n_jobs=8)
y_pred = kmeans.fit_predict(X_embedded)

In [5]:
output_notebook()

y_labels = y_pred

# data sources
source = ColumnDataSource(data=dict(x= X_embedded[:,0], y= X_embedded[:,1], x_backup = X_embedded[:,0],
                                    y_backup = X_embedded[:,1], desc= y_labels, descriptions= working_experiences['description'],
                                    employers = working_experiences['employer'], keywords = working_experiences['keyword']
                                    ))

# map colors
mapper = linear_cmap(field_name='desc', palette=RdPu9,low=min(y_labels), high=max(y_labels))

# hover over information
hover = HoverTool(tooltips=[
    ("Employer", "<b>@employers{safe}</b>"),
    ("Keyword", "<b>@keywords{safe}</b>"),
    ("Description", "@descriptions{safe}")
    ],
    point_policy="follow_mouse")

p = figure(plot_width=550, plot_height=550, tools=[hover], title="Cluster of all Working Experiences", toolbar_location=None)

# plot
p.scatter('x', 'y', size=10, source=source, fill_color=mapper, line_alpha=0.3, line_color="black")

labels = LabelSet(x="x", y="y", text="keywords", y_offset=-20, text_font_size="9pt", text_color="#555555",
                      source=source, text_align='center')
p.add_layout(labels)

work_labels = ['All'] + list(working_experiences.employer.unique())

option = RadioButtonGroup(labels=work_labels, active=0)
option.js_on_click(CustomJS(args=dict(p=p, source=source), code="""
    var radio_value = cb_obj.active;
    var x = source.data.x
    var y = source.data.y
    var x_orig = source.data.x_backup
    var y_orig = source.data.y_backup
    var employers = source.data.employers

    for (var i = 0; i < x.length; i++) {
        if(employers[i] == cb_obj.labels[radio_value] || cb_obj.labels[radio_value] == 'All') {
            x[i] = x_orig[i]
            y[i] = y_orig[i]
        } else {
            x[i] = undefined
            y[i] = undefined
        }
    }
    
    p.title.text = 'Cluster of Working Experiences in ' + cb_obj.labels[radio_value]
    
    source.change.emit();

"""))

checkbox_group = CheckboxGroup(labels=['Labels'], active=[0])
checkbox_group.js_on_click(CustomJS(args=dict(labels=labels),code="""
    if(this.active == '0') {
        labels.visible = true
        console.log('true')
    } else {
        labels.visible = false
    }
    labels.change.emit();
"""))

show(column(column(column(option),checkbox_group),p))

Loading BokehJS ...

_________________
### February 2020 - Present, Innovation Area Manager
<img src="cedacri.png" width=50 align="right">

#### Cedacri, Parma
**Business or sector** Research & Development, Data & Analytics, Machine Learning, Cloud

_________________
### February 2019 - February 2020 (1 year), Senior Engineer ML - Data Scientist
<img src="myinvenio.png" width=50 align="right">

#### MyInvenio, Reggio Emilia
**Business or sector** Data Science, Machine Learning, Process Discovery & Mining

_________________
### March 2015 - February 2019 (3 years 11 months), Computer Engineer – Software Development Leader
<img src="sia.png" width=50 align="right">

#### Ubiq S.R.L. - SIA Group, Parma/Milan
**Business or sector** Computer Engineering, Big Data, Machine Learning

_________________
### August 2014 - February 2015 (7 months), Software Engineering Intern
<img src="datalogic.png" width=90 align="right">

#### Datalogic ADC Inc, 55 W Del Mar Blvd, Pasadena (CA)

**Business or sector** Computer Vision

__________________
## EDUCATION
### October 2017 - Present, Member of the <a href="https://www.dsaiparma.it/" target="_blank">Data Science and AI group in Parma</a>
<img src="dsai.jpeg" width=70 align="right">

#### Parma

* Speaker at public events on Machine Learning and Data Science topics
______________
### September 2012 - March 2015, Master's Degree in Computer Engineering QEQ7
<img src="https://www.unibo.it/it/logo-unibo.png" width=90 align="right">

#### University of Bologna, Bologna

* Computer vision, data mining, use of Hadoop framework, information security, artificial intelligence, computer networks, mobile systems, OpenCV, Java, C/C++, VHDL, IT, Real-Time systems.

**Thesis** *Development of parallel processing approach to compute Zero-mean Normalized Cross- Correlation. CUDA framework has been used to develop fast approach to detect characters in images, using Template Matching approach, based on Zero-mean Normalized Cross-Correlation measure.*

**Final Grade** 110/110 with Honours
______________
### September 2009 - July 2012, Bachelor's Degree in Computer Engineering QEQ6
<img src="https://www.unipr.it/sites/default/files/styles/paragrafo/public/contenuto_generico/immagini/unipr_centrato_1riga_pos_rgb.jpg?itok=SkbDQ2gI" width=90 align="right">

#### University of Parma, Parma
* Computer science, mathematics, physic, industrial robotics, web programming, electronics, automatics, computer graphics, software engineering, Java, C/C++, OpenGL

**Thesis** *A Machine Learning project based on arm gesture recognition and humanoid imitation. Arm gestures are acquired by inertial motion sensors and modelled in an OpenGL 3D virtual space. A new method based on Functional Principal Component Analysis is used in MATLAB for both a supervised clustering of training data and gesture recognition. Recognized gestures are imitated by a small humanoid robot*

**Final grade** 107/110

______________
## PERSONAL SKILLS
### LANGUAGES

  
| Language | Listening | Reading | Spoken Interacion | Spoken Production | Writing |
| -------- | --------- | ------- | ----------------- | ----------------- | ------- |
| Italian | MT | MT | MT | MT| MT |
| English | C2 | C2 | C2 | C2 | C1 |
| German | A1 | A1 | A1 | A1 | A1 |

*Levels: A1/2: Basic user - B1/2: Independent user - C1/2 Proficient user - MT Mother Tongue*

### COMMUNICATION SKILLS
I gained good communication skills speaking and interacting with customers and clents. I'm also part of the Data Science and Artificial Intelligence Group in Parma, and I give a speech about Machine Learning topics once every few months, usually in front of dozens of peoples.

### ORGANISATIONAL/MANAGERIAL SKILLS
Excellent social/organisational skills acquired during my experience as Team Leader/Area Manager. 

### JOB-RELATED SKILLS

In [6]:
def show_skills_plot(x_label, skills=None):
    if skills is None:
        raise Exception("you should have at least one skill, shouldn't you?")
    from bokeh.io import show, output_notebook
    from bokeh.plotting import figure
    from bokeh.palettes import RdPu9
    from bokeh.transform import factor_cmap
    from bokeh.models import ColumnDataSource
    output_notebook()

    
    source = ColumnDataSource(skills)
    
    TOOLTIPS = """
        <div style="width:300px">
            <table>
                <tr><td style="color:Fuchsia;">Name</td><td style="text-align: center;"><b>@description</b></td></tr>
                <tr><td style="color:Fuchsia;">Type</td><td style="text-align: center;">@subtype</td></tr>
                <tr><td style="color:Fuchsia;">Details</td><td style="text-align: center;">@tooltip</td></tr>
            </table>
        </div>
    """

    p = figure(plot_width=700, plot_height=400, title="Expertise level",
               x_range=list(skills.description), toolbar_location=None, tooltips=TOOLTIPS)

    index_cmap = factor_cmap('subtype', palette=RdPu9, factors=sorted(skills.subtype.unique()), end=1) 
    p.vbar(x='description', top='level', width=1, source=source,
           line_color="white", fill_color=index_cmap)

    p.y_range.start = 0
    p.y_range.end = 10
    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.xaxis.axis_label = x_label
    p.xaxis.major_label_orientation = 1.2
    p.outline_line_color = None

    show(p)

job_skills = pd.read_csv('job_skills.csv', sep=';')
job_skills['subtype'] = job_skills.apply(lambda x : x.type if pd.isna(x.subtype) else x.subtype, axis=1)

**Programming Languages**

In [7]:
show_skills_plot('Programming Languages', job_skills[job_skills.type == 'Programming Language'])

Loading BokehJS ...

**Java & DevOps**

In [8]:
show_skills_plot('Skills grouped by type', job_skills[job_skills.type.isin(['Java','DevOps'])].sort_values(by = ['subtype','level'], ascending=False))

Loading BokehJS ...

**Containers**

In [9]:
show_skills_plot('Skills grouped by type', job_skills[job_skills.type.isin(['Containers'])].sort_values(by = ['subtype','level'], ascending=False))

Loading BokehJS ...

**Big Data**

In [10]:
show_skills_plot('Skills grouped by type', job_skills[job_skills.type.isin(['Big Data'])].sort_values(by = ['subtype','level'], ascending=False))

Loading BokehJS ...

**Other job-related skills**

In [11]:
show_skills_plot('Skills grouped by type', 
                 job_skills[job_skills.type.isin([key for key in list(job_skills.type.unique()) if key not in ['Java', 'DevOps', 'Containers', 'Programming Language']])])

Loading BokehJS ...

### OTHER SKILLS
**Sport**: 
* American Football, 4 years’ experience in the Parma Panthers, interaction with American staff and players. Participation in European Championship with the Italian National American Football team

**Driving License**: B (own car)
______________
## ADDITIONAL INFORMATION
**Publications**

[Arm Gesture Recognition and Humanoid Imitation Using Functional Principal Component Analysis](https://ieeexplore.ieee.org/abstract/document/6696892),<br>
J. Aleotti, A. Cionini, L. Fontanili, S. Caselli,<br>
IEEE/RSJ International Conference on Intelligent Robotics and Systems (IROS),<br>
Tokyo, November 2013

**Speaking**

**[BPMNext 2019](https://youtu.be/cBeZX_S4vGo)**: Process Mining and DTO — How to Derive Business Rules and ROI from the Data

**Honours and awards**

Bronze medal of sporting merit given by CONI

**Certifications**

* Process Mining: Data science in Action, Eindhoven University of Technology [Coursera, #R95WTVQ56CQS](https://www.coursera.org/account/accomplishments/verify/R95WTVQ56CQS)
* Machine Learning, Stanford University, [Coursera, #FGYRSK7XD8XG](https://www.coursera.org/account/accomplishments/verify/FGYRSK7XD8XG)
* Using Python for Research, Harvard University, [edX, #f079493d7801420388179e219e2a9d63](https://courses.edx.org/certificates/f079493d7801420388179e219e2a9d63)

**Projects**

* **[myInvenio](https://www.my-invenio.com)**: A Process Mining tool to gather insightful from business processes
* Development of projects of personal interest using latest mobile and web programming languages (Android, HTML, CSS, SQL, JavaScript, Python) and tools (Nutch crawler, SonarQube, Tesseract).
* **[Ti Frutta](https://www.tifrutta.it)**: The very first "cash back" app in Italy that allows the customer to earn by shopping
* **[F Abbigliamento](https://www.fontaniliabbigliamento.it)**: development of the Fontanili Abbigliamento website using Wordpress CRM
* **CV – Eye Detector**: development of an eye detector in C++ based on Normalized Cross Correlation, robust to light variation
* **DSS – Hadoop**: implementation in Apache Hadoop environment of the Distributed Solving Set algorithm for outlier detection in large distributed data sets, using the MapReduce model
* **Parma Panthers Official**:  development of the Android official application of the Parma Panthers, the American Football team 4 times champion of Italy in the main championship
* **PicoPic**: VHDL project of a Peripheral Interface Controller for a stereo vision system on FPGA, with a 8 bit RISC microprocessor PicoBlaze
* **Wikiquote Search Engine**: implementation of a web crawler for Wikiquote based on Apache Nutch and development of a Java search engine based on Apache Solr
* **[pysqoop](https://pypi.org/project/pysqoop/)**: a Python package that lets you sqoop into HDFS data from RDBMS using Apache Sqoop (installable via pip)
* Member & Speaker of the Data Science & AI Group in Parma

______________
## STUDIES
### BOOKS

In [12]:
pd.read_sql_query('SELECT * FROM books order by status desc, author asc, title asc', conn)

,title,author,status
0,Mastering Java Machine Learining,"Kamath, Uday",reading
1,"The Art of Computer Programming, Fundamental Algorithms","Knuth, Donald",reading
2,Test-Driven Development,"Beck, Kent",read
3,Effective Java,"Bloch, Josua",read
4,Design Patterns,"Gamma, Helm, Johnson, Vlissides",read
5,97 Things Every Programmer Should Know,"Henney, Kevlin",read
6,Agile Software Development,"Martin, Robert C.",read
7,Clean Code,"Martin, Robert C.",read
8,The Clean Coder,"Martin, Robert C.",read
9,Decision Model and Notation,OMG,read


### COURSES

In [13]:
pd.read_sql_query('SELECT * FROM courses order by status desc, platform desc, held_by asc, title asc', conn)

,title,held_by,platform,status
0,Analytics in Python,Columbia,edX,completed
1,Artificial Intelligence,Columbia,edX,completed
2,Machine Learning,Columbia,edX,completed
3,Robotics,Columbia,edX,completed
4,Using Python for Research,Harvard,edX,completed
5,Python for Data Science,UC San Diego,edX,completed
6,Blockchain for Business,Udemy,Udemy,completed
7,Certified Kubernetes Application Developer (CKAD),Sighup,Sighup,completed
8,Blockchain – Principles and Practices,Pluralsight,Pluralsight,completed
9,Building Your First scikit-learn Solution,Pluralsight,Pluralsight,completed


______________
## MY DAY

In [14]:
from math import pi

import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.palettes import PuRd7
from bokeh.plotting import figure
from bokeh.transform import cumsum

output_notebook()

x = {
    'Sleep': 6.5/24,
    'Work': 9/24,
    'Gym': 1.5/24,
    'Study': 2/24,
    'Other': 5/24
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'activity'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = PuRd7[:len(x)]

p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@activity: @value%", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='activity', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

show(p)


Loading BokehJS ...

### GET IN TOUCH
Feel free to contact me at:

<center><a href="mailto:luca.fontanili93@gmail.com"><i class="fa fa-envelope"></i>luca.fontanili93@gmail.com</a></center>

<center><b>This is still a work in progress</b></center>
<img src="spinner.gif" width=200 align="center">